In [66]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 5: Alchemical Reduction ---</h2><p>You've managed to sneak in to the prototype suit manufacturing lab.  The Elves are making decent progress, but are still struggling with the suit's size reduction capabilities.</p>
<p>While the very latest in 1518 alchemical technology might have solved their problem eventually, you can do better.  You scan the chemical composition of the suit's material and discover that it is formed by extremely long <a href="https://en.wikipedia.org/wiki/Polymer">polymers</a> (one of which is <span title="I've always wanted a polymer!">available</span> as your puzzle input).</p>
<p>The polymer is formed by smaller <em>units</em> which, when triggered, react with each other such that two adjacent units of the same type and opposite polarity are destroyed. Units' types are represented by letters; units' polarity is represented by capitalization.  For instance, <code>r</code> and <code>R</code> are units with the same type but opposite polarity, whereas <code>r</code> and <code>s</code> are entirely different types and do not react.</p>
<p>For example:</p>
<ul>
<li>In <code>aA</code>, <code>a</code> and <code>A</code> react, leaving nothing behind.</li>
<li>In <code>abBA</code>, <code>bB</code> destroys itself, leaving <code>aA</code>.  As above, this then destroys itself, leaving nothing.</li>
<li>In <code>abAB</code>, no two adjacent units are of the same type, and so nothing happens.</li>
<li>In <code>aabAAB</code>, even though <code>aa</code> and <code>AA</code> are of the same type, their polarities match, and so nothing happens.</li>
</ul>
<p>Now, consider a larger example, <code>dabAcCaCBAcCcaDA</code>:</p>
<pre><code>dabA<em>cC</em>aCBAcCcaDA  The first 'cC' is removed.
dab<em>Aa</em>CBAcCcaDA    This creates 'Aa', which is removed.
dabCBA<em>cCc</em>aDA      Either 'cC' or 'Cc' are removed (the result is the same).
dabCBAcaDA        No further actions can be taken.
</code></pre>
<p>After all possible reactions, the resulting polymer contains <em>10 units</em>.</p>
<p><em>How many units remain after fully reacting the polymer you scanned?</em> <span class="quiet">(Note: in this puzzle and others, the input is large; if you copy/paste your input, make sure you get the whole thing.)</span></p>
</article>


In [67]:
def react(polymer: str) -> int:
    p = list(polymer.strip())
    i = 0

    while i < len(p) - 1:
        if reacts(p, i):
            p.pop(i)
            p.pop(i)
            if i > 0:
                i -= 1
        else:
            i += 1
    return len(p)


def reacts(p, i):
    return p[i].lower() == p[i + 1].lower() and (
        (p[i].islower() and p[i + 1].isupper())
        or (p[i].isupper() and p[i + 1].islower())
    )


tests = [
    {
        "name": "Example 1",
        "polymer": "aA",
        "expected": 0,
    },
    {
        "name": "Example 2",
        "polymer": "abBA",
        "expected": 0,
    },
    {
        "name": "Example 3",
        "polymer": "abAB",
        "expected": 4,
    },
    {
        "name": "Example 4",
        "polymer": "aabAAB",
        "expected": 6,
    },
    {
        "name": "Example 5",
        "polymer": "dabAcCaCBAcCcaDA",
        "expected": 10,
    },
]

run_tests_params(react, tests)


Test Example 1 passed, for react.
Test Example 2 passed, for react.
Test Example 3 passed, for react.
Test Example 4 passed, for react.
Test Example 5 passed, for react.
Success


In [68]:
with open("../input/day5.txt") as f:
    puzzle = f.read()

print(f"Part I: {react(puzzle)}")

Part I: 10384


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>10384</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Time to improve the polymer.</p>
<p>One of the unit types is causing problems; it's preventing the polymer from collapsing as much as it should.  Your goal is to figure out which unit type is causing the most problems, remove all instances of it (regardless of polarity), fully react the remaining polymer, and measure its length.</p>
<p>For example, again using the polymer <code>dabAcCaCBAcCcaDA</code> from above:</p>
<ul>
<li>Removing all <code>A</code>/<code>a</code> units produces <code>dbcCCBcCcD</code>. Fully reacting this polymer produces <code>dbCBcD</code>, which has length 6.</li>
<li>Removing all <code>B</code>/<code>b</code> units produces <code>daAcCaCAcCcaDA</code>. Fully reacting this polymer produces <code>daCAcaDA</code>, which has length 8.</li>
<li>Removing all <code>C</code>/<code>c</code> units produces <code>dabAaBAaDA</code>. Fully reacting this polymer produces <code>daDA</code>, which has length 4.</li>
<li>Removing all <code>D</code>/<code>d</code> units produces <code>abAcCaCBAcCcaA</code>. Fully reacting this polymer produces <code>abCBAc</code>, which has length 6.</li>
</ul>
<p>In this example, removing all <code>C</code>/<code>c</code> units was best, producing the answer <em>4</em>.</p>
<p><em>What is the length of the shortest polymer you can produce</em> by removing all units of exactly one type and fully reacting the result?</p>
</article>

</main>


In [85]:
example = "dabAcCaCBAcCcaDA"


def shortest_possible_polymer(polymer: str) -> int:
    p = polymer.strip()
    letters = set(map(str.lower, p))
    return min(react(p.replace(l, "").replace(l.upper(), "")) for l in letters)


print(f"Example: {shortest_possible_polymer(example)} should be 4")

Example: 4 should be 4


In [86]:
print(f"Part II: {shortest_possible_polymer(puzzle)}")  # 5412

Part II: 5412


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>5412</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
